## Receiver operating characteristic curves

Suppose that we have a binary classification task, in which we have to map a 'sample' $\mathbf{x}$ to a class $y \in \{ 0, 1\}$. We may use a machine learning method to learn a function $\hat{y} = h(\mathbf{x})$. 

How can we determine how good a binary classifier is at its task? 

One way in which one could do this is to simply take the percentage of times that the classifier is correct. Problem: _skewed_ 

In order to deal with this, ... FP TP
But, how can we compare it with another classifier? 

Often, $h(\mathbf{x})$ will not be a binary function itself. For instance, if $h$ is a neural network with a sigmoid activation function 




## References

[1] Fawcett, T. (2006). An introduction to ROC analysis. Pattern recognition letters, 27(8), 861-874. – Sections 1-5 are relevant to the course.